In [1]:
from torchvision import datasets
import torch
import torchvision.transforms as tf
import pdb
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [2]:
from MyNeuralNetwork import NeuralNetwork

In [3]:
def GetDatasets(download=True):
    dataset_train = datasets.FashionMNIST(
        root = r'data/train',
        download = download,
        train = True,
        transform = tf.ToTensor() 
    )
    
    dataset_test = datasets.FashionMNIST(
        root = r'data/test',
        download = download,
        train = False,
        transform = tf.ToTensor() 
    )
    return dataset_train, dataset_test

In [4]:
dataset_train, dataset_test = GetDatasets(False);

In [5]:
batch_size = 500
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

In [6]:
device = 'cpu'
network = NeuralNetwork().to(device)

In [7]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=0.0001)

In [8]:
num_epochs = 25

In [9]:
for lr in [0.5, 0.05, 0.005, 0.0005]:
    optimizer = torch.optim.SGD(network.parameters(), lr=lr)
    
    log_dir = './logs/{}'.format(lr)
    writer = SummaryWriter(log_dir=log_dir)

    for epoch in range(num_epochs):
        
        epoch_loss = 0
        epoch_accuracy = 0
        
        for idx, batch in enumerate(dataloader_train):
            # zero the gradient
            optimizer.zero_grad()

            x, y = batch[0], batch[1]

            # flatten it first
            x = torch.flatten(x, start_dim=1)

            # forward pass
            predictions = network(x)

            loss = criterion(predictions, y)

            # backpropogate the loss
            loss.backward()

            # step the optimizer in the direction of the gradient
            optimizer.step()
            epoch_loss += loss.item()

        # evaluate prediction accuracy in test set
        for idx, batch in enumerate(dataloader_test):
            network.eval()
            
            x, y = batch[0], batch[1]
            x = torch.flatten(x, start_dim=1)
            predcitions = network(x)
            
            pred_label = torch.argmax(predictions, dim=1)
            n_correct = (y==pred_label).float()
            accuracy = n_correct.sum() / n_correct.numel()
            epoch_accuracy += 100 * accuracy
            
        # tensorboard 
        writer.add_scalar('Epoch loss', epoch_loss / idx, epoch)
        writer.add_scalar('Epoch accuracy', epoch_accuracy / idx, epoch)
        writer.flush()
        
        print('Learning rate: {:>7}, Epoch: {}, Training loss: {:.2f}, Test accuracy : {:.2f}%'.format(
            lr, epoch + 1, epoch_loss / idx, epoch_accuracy / idx))
        writer.close()


Learning rate:     0.5, Epoch: 1, Training loss: 12.86, Test accuracy : 10.86%
Learning rate:     0.5, Epoch: 2, Training loss: 11.18, Test accuracy : 10.68%
Learning rate:     0.5, Epoch: 3, Training loss: 10.74, Test accuracy : 9.93%
Learning rate:     0.5, Epoch: 4, Training loss: 10.62, Test accuracy : 9.94%
Learning rate:     0.5, Epoch: 5, Training loss: 10.55, Test accuracy : 10.47%
Learning rate:     0.5, Epoch: 6, Training loss: 10.51, Test accuracy : 9.97%
Learning rate:     0.5, Epoch: 7, Training loss: 10.48, Test accuracy : 10.02%
Learning rate:     0.5, Epoch: 8, Training loss: 10.46, Test accuracy : 10.43%
Learning rate:     0.5, Epoch: 9, Training loss: 10.43, Test accuracy : 10.65%
Learning rate:     0.5, Epoch: 10, Training loss: 10.42, Test accuracy : 10.77%
Learning rate:     0.5, Epoch: 11, Training loss: 10.42, Test accuracy : 10.35%
Learning rate:     0.5, Epoch: 12, Training loss: 10.39, Test accuracy : 10.18%
Learning rate:     0.5, Epoch: 13, Training loss: 10

In [ ]:
#% tensorboard --logdir=logs --host=127.0.0.1